# Project 3 Data Cleaning: Sharks attacks



## Modules used

In [1]:
import pandas as pd
import numpy as np
import re
import itertools

## Dataset import and exploration

In [2]:
df = pd.read_csv('./attacks.csv', encoding='ISO-8859-1')
df_clean = df.copy()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [3]:
def check_nulls():
    return df_clean.isnull().mean() * 100
check_nulls()

Case Number               66.170353
Date                      75.500525
Year                      75.508300
Type                      75.516075
Country                   75.694903
Area                      77.269370
Location                  77.599813
Activity                  77.615364
Name                      76.316915
Sex                       77.697003
Age                       86.506240
Injury                    75.609377
Fatal (Y/N)               77.595926
Time                      88.539439
Species                   86.533453
Investigator or Source    75.566614
pdf                       75.500525
href formula              75.504412
href                      75.500525
Case Number.1             75.500525
Case Number.2             75.500525
original order            75.473312
Unnamed: 22               99.996112
Unnamed: 23               99.992225
dtype: float64

## Changes on columns name to avoid whitespaces.



In [4]:
df_clean.columns = df_clean.columns.str.strip().str.lower()
df_clean.columns = df_clean.columns.str.replace(" ","_")
df_clean.columns






Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number.1', 'case_number.2', 'original_order', 'unnamed:_22',
       'unnamed:_23'],
      dtype='object')

## Nulls columns cleanse:
### Since  the dataset have columns Unnamed:22 and Unnamed: 23 with almost entirely null values, they are ment to be dropped.  

In [5]:
null_columns = df_clean.isnull().mean() * 100
null_columns = null_columns[null_columns > 99]
null_columns_eliminate = list(null_columns.index)
df_clean = df_clean.drop(null_columns_eliminate, axis=1)

### Due to country, area and location columns have a large portion of null data. The efforts to reduce data were aimed in reduce as many rows as possible with null data


In [6]:
df_clean[['country','area', 'location']].isna()

,country,area,location
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
25718,True,True,True
25719,True,True,True
25720,True,True,True
25721,True,True,True


In [7]:
cleaning_country_area_location = df_clean[(df_clean['country'].isna()) & (df_clean['area'].isna()) & (df_clean['location'].isna())].index
df_clean = df_clean.drop(cleaning_country_area_location, axis=0)

In [8]:
only_area = (df_clean['location'].isna()) & (df_clean['country'].isna())
only_country = (df_clean['location'].isna()) & (df_clean['area'].isna())
only_location = (df_clean['country'].isna()) & (df_clean['area'].isna())
excluding_area = (df_clean['area'].isna())
filter_cleaned = df_clean[(only_area) | (only_country) | (only_location) | (excluding_area) ].index

df_clean = df_clean.drop(filter_cleaned, axis=0)
df_clean[df_clean['country'].isna()]

,case_number,date,year,type,country,area,location,activity,name,sex,...,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order
3387,1970.08.02,02-Aug-1970,1970.0,Invalid,NaN,Caribbean Sea,Between St. Kitts & Nevis,Sea Disaster Sinking of ferryboat Christina,NaN,NaN,...,NaN,Afternoon,Shark involvement prior to death was not confi...,"Rome News Tribune, 8/3/1970",1970.08.02-Christina-ferryboat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1970.08.02,1970.08.02,2916.0
4266,1956.00.00.g,1956,1956.0,Sea Disaster,NaN,Between Comores & Madagascar,Geyser Bank,Shipwreck,"Captain Eric Hunt, the cook & a French passenger",M,...,Y,NaN,NaN,dinofish.com,1956.00.00.g-Capt-Hunt.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1956.00.00.g,1956.00.00.g,2037.0
4498,1949.12.00.b,Dec-1949,1949.0,Sea Disaster,NaN,Caribbean Sea,Between Cuba & Costa Rica,"Sea Disaster, sinking of the motorship Wingate","Albert Battles, James Dean & 4 crew",M,...,Y,NaN,Shark involvement not confirmed,"Canberra Times, 1/6/1950",1949.12.00.b-Wingate.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1949.12.00.b,1949.12.00.b,1805.0
5020,1931.04.27.R,Reported 27-Apr-1931,1931.0,Unprovoked,NaN,French Southern Territories,Île Saint-Paul,"Fishing, boat capsized",Quillezic,M,...,Y,NaN,NaN,"Los Angeles Times, 4/27/1931",1931.04.27.R-Quillezic.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1931.04.27.R,1931.04.27.R,1283.0
6137,1787.07.05,05-Jul-1787,1787.0,Unprovoked,NaN,St Helena,Landing Place,Swimming,Private Isaac Hicksled,M,...,Y,NaN,NaN,"H.R. Janisch (1885), Extracts from the St. Hel...",1787.07.05-Hicksled.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1787.07.05,1787.07.05,166.0


### Since we still have countries as null values, we might infer their names with the information provided.

In [9]:
countries = {3387:'St. Kitts Nevis', 4266:'Madagascar', 4498:'Cuba', 5020:'France', 6137:'Cabo Verde'}

for index,country in countries.items():
    df_clean.at[index, 'country'] = country

df_clean[df_clean['country'].isna()]

,case_number,date,year,type,country,area,location,activity,name,sex,...,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order


### If we take a deep look on the age of the victims we can see some information we might want to drop due to inconsistencies and on the other hand we have some information we can still infer.

In [10]:
print(df_clean['age'].unique())
Age_list = ['Teen','28 & 26','46 & 34','28, 23 & 30','Teens','36 & 26','Â','6Â½','21 & ?','23 & 20','7      &    31','Elderly',
'Ca. 33','>50','adult','9 & 12','? & 19','23 & 26','(adult)','33 & 37','37, 67, 35, 27,  ? & 27','21, 34,24 & 35','30 & 32',
'50 & 30','17 & 35','X','"middle-age"','34 & 19','33 & 26','2 to 3 months','MAKE LINE GREEN','"young"','17 & 16','F','young','36 & 23',
'A.M.','?    &   14','2Â½','teen','Â','\xa0 ', ' ', '  ']

index_age=[]
unique_age=[]
for i in df_clean['age']:
    if i in Age_list:
        if i not in unique_age:
            index_age.append(list(df_clean.index[df_clean['age']==i]))
        unique_age.append(i)

index_age = list(itertools.chain.from_iterable(index_age))

df_clean= df_clean.drop(index_age, axis=0)

['57' '11' '48' nan '18' '52' '15' '12' '32' '10' '21' '34' '30' '60' '33'
 '29' '54' '41' '37' '56' '19' '25' '69' '38' '55' '35' '46' '45' '14'
 '28' '20' '24' '26' '49' '22' '7' '31' '17' '40' '13' '42' '3' '8' '50'
 '16' '82' '73' '20s' '68' '51' '39' '58' 'Teen' '47' '61' '40s' '65' '36'
 '66' '43' '60s' '9' '72' '59' '6' '27' '64' '23' '71' '44' '62' '63' '70'
 '18 months' '53' '30s' '50s' 'teen' '74' '28 & 26' '5' '86' '18 or 20'
 '12 or 13' '46 & 34' '28, 23 & 30' 'Teens' '77' '36 & 26' '8 or 10' '84'
 '\xa0 ' ' ' '30 or 36' '6½' '21 & ?' '75' '33 or 37' 'mid-30s' '23 & 20'
 ' 30' '7      &    31' ' 28' "60's" '16 to 18' '87' '67' 'Elderly'
 'mid-20s' 'Ca. 33' '74 ' '45 ' '21 or 26' '20 ' '>50' '18 to 22' 'adult'
 '9 & 12' '? & 19' '9 months' '23 & 26' '1' '(adult)' '33 & 37' '25 or 28'
 '37, 67, 35, 27,  ? & 27' '21, 34,24 & 35' '30 & 32' '50 & 30' '17 & 35'
 'X' '"middle-age"' '13 or 18' '34 & 19' '33 & 26' '2 to 3 months'
 'MAKE LINE GREEN' ' 43' '81' '"young"' '7 or 8' '78'

In [11]:
age_dict={"20s" : 25, "40s" : 45,"60s": 65,"18 months": 2,"30s" : 35,"50s" : 55,"18 or 20" : 19,"12 or 13" : 13,
               "8 or 10": 9,"30 or 36" : 33,"33 or 37" : 35,"mid-30s" : 35,"60's" : 65, "16 to 18" : 17,"mid-20s" : 25,
               "21 or 26" : 24, "18 to 22" : 20,"9 months" : 1,"25 or 28" : 27,"13 or 18" : 15, "7 or 8": 8,"Both 11" : 11,
               "9 or 10" : 10,"10 or 12": 11,"31 or 33" : 32,"13 or 14" : 13,'6½': 6,' 30': 30,' 28': 28,'74 ':74, '45 ':45,
              '20 ':20, ' 43':43, '2½':2}

df_clean['age'] = df_clean['age'].replace(age_dict)

## Just to add we need to change the age type of value to integer

In [12]:
df_clean['age'].unique()
df_clean['age'] = np.floor(pd.to_numeric(df_clean['age'], errors='coerce')).astype('Int64')
df_clean['age'].unique()

<IntegerArray>
[  57,   11,   48, <NA>,   18,   52,   15,   12,   32,   10,   21,   34,   30,
   60,   33,   29,   54,   41,   37,   56,   19,   25,   69,   38,   55,   35,
   46,   45,   14,   28,   20,   24,   26,   49,   22,    7,   31,   17,   40,
   13,   42,    3,    8,   50,   16,   82,   73,   68,   51,   39,   58,   47,
   61,   65,   36,   66,   43,    9,   72,   59,    6,   27,   64,   23,   71,
   44,   62,   63,   70,    2,   53,   74,    5,   86,   77,   84,   75,   87,
   67,    1,   81,   78]
Length: 82, dtype: Int64

### Injury column have some null data to take care on.

In [13]:
injuried_cleaned = df_clean[df_clean['injury'].isna()]
index_list = injuried_cleaned.index.values.tolist()
df_clean= df_clean.drop(index_list, axis=0)
df_clean

,case_number,date,year,type,country,area,location,activity,name,sex,...,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6296,ND.0006,Before 1906,0.0,Unprovoked,AUSTRALIA,New South Wales,,Swimming,Arab boy,M,...,Y,NaN,Said to involve a grey nurse shark that leapt ...,"L. Becke in New York Sun, 9/9/1906; L. Schultz...",ND-0006-ArabBoy-Prymount.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0006,ND.0006,7.0
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0


In [14]:
def parse_injuries(row):
    string_row = str(row)
    string_row = string_row.strip().lower()
    if string_row == 'y':
        return True
    
    elif string_row == 'fatal' or string_row =='n' or string_row == 'm' or string_row == 'unknown' or string_row == '2017':
        return False
    
    else:
        return row


df_clean['fatal_(y/n)'] = df_clean['fatal_(y/n)'].apply(parse_injuries)
df_clean['fatal_(y/n)'].value_counts()

False    4110
True     1178
Name: fatal_(y/n), dtype: int64

### We still have some values to cover so the following words were inferred to change the values of some rows

In [15]:
df_clean.reset_index(inplace=True, drop=True)
key_words =['mortem','killed','Human','human', 'death','Death','disap','Disap','drown','Drown','Missing','missing',
'recovered','Forensic','body','bodies','corpse','decap','Decap','perish','remains']

pil=[]
j=0
for word in key_words:
    j = 0
    for i in df_clean['injury']:
        if word in i:
            pil.append(j)
            df_clean['fatal_(y/n)'][j] = True
        j = j + 1
        
pil_listset=list(set(pil))

df_clean['fatal_(y/n)'].fillna(False, inplace = True)

/var/folders/dm/mxz6j4h90psdrp7fypj_85vm0000gn/T/ipykernel_3503/4250053987.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['fatal_(y/n)'][j] = True


### Now we need to check on date column to do a cleanse process extracting the values from other columns and inferring from case number columns, and parsing dates

#### Cleaning dates pattern . for / 

In [16]:
df_clean['date2'] = df_clean['date']
df_clean['date3'] = df_clean['case_number']

def cast_date(date):
    date = str(date).strip()
    date = date[:10].replace('.','-')
    if date.endswith('00'):
        date = list(date)
        date[-1] = '1'
        return  "".join(date)
    return date
    
df_clean['date'] = df_clean['case_number'].apply(cast_date)

#### Verifying columns on Case Number and Dates, in order to infer years

In [17]:
nd_values = df_clean[df_clean['case_number'].str.contains('ND', na=False)]
index = df_clean[df_clean['case_number'].str.contains('ND', na=False)].index
index = list(index)
nd_values

for i in index:
    if any(chr.isdigit() for chr in df_clean['date2'][i] )== False:
        df_clean['date2'][i] = None
    
    else:   
         df_clean['date2'][i] = re.findall(r'[0-9]{4}',df_clean['date2'][i])[0]

/var/folders/dm/mxz6j4h90psdrp7fypj_85vm0000gn/T/ipykernel_3503/943555934.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['date2'][i] = re.findall(r'[0-9]{4}',df_clean['date2'][i])[0]
/var/folders/dm/mxz6j4h90psdrp7fypj_85vm0000gn/T/ipykernel_3503/943555934.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['date2'][i] = None


#### Dropping columns with biased dates 

In [18]:
df_clean[df_clean['date2'].isna()].index
df_clean = df_clean.drop(df_clean[df_clean['date2'].isna()].index, axis=0)

biased_elder_ages = df_clean[(df_clean['date'].str.startswith('000')) | (df_clean['date'] == 'nan')].index
df_clean = df_clean.drop(biased_elder_ages, axis=0)

#### Doing cleanse on some isolated cases

In [19]:
def parse_date(row):
    row = str(row)
    if row[4:8] == '-00-':
        row = list(row)
        row[6] = '1'
        return  "".join(row)
    else:
        return row
        

df_clean['date'] = df_clean['date'].apply(parse_date)

# Resetting index
df_clean.reset_index(inplace=True, drop=True)

In [20]:
index = df_clean[df_clean['case_number'].str.contains('ND', na=False)].index
index = list(index)
for i in index:
    df_clean['date'][i] = df_clean['date2'][i]
    
df_clean['date'] = df_clean['date'].str.replace(',', '-')

df_clean['date'] = pd.to_datetime(df_clean['date'], errors='coerce')
dates_index = df_clean[df_clean['date'].isna()].index
df_clean = df_clean.drop(dates_index, axis=0)

/var/folders/dm/mxz6j4h90psdrp7fypj_85vm0000gn/T/ipykernel_3503/2555339065.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['date'][i] = df_clean['date2'][i]


### It's time to drop some columns helpers and nullish columns that won't help our further research

In [21]:
df_clean.drop(["case_number.1", "case_number.2", "case_number", "year", "original_order", "date2", "date3", "time"], axis=1, inplace=True)

### The column 'name' of the vicitims won't be part of the core of the further analysis. We don't have enough information to infer names, so will fill the values as anonymous.

In [22]:
df_clean['name'] = df_clean['name'].fillna('Anonymous')


### There is no much to do on gender column. Just changing the values

In [23]:
df_clean['sex'].info()

<class 'pandas.core.series.Series'>
Int64Index: 5745 entries, 0 to 5748
Series name: sex
Non-Null Count  Dtype 
--------------  ----- 
5261 non-null   object
dtypes: object(1)
memory usage: 89.8+ KB


In [24]:
df_clean['sex'] = df_clean['sex'].str.strip()
df_clean['sex'] = df_clean['sex'].fillna('Unknown')

def gender_identifier(row):
    identifiers = ['M', 'F', 'Unknown']
    if row not in identifiers:
        row = 'Unknown'
        return row
    else:
        return row


df_clean['sex'] = df_clean['sex'].apply(gender_identifier)

df_clean['sex'].value_counts()

M          4649
F           609
Unknown     487
Name: sex, dtype: int64

### As well in Investigation or Source column, so I decided to drop some values that doesn't seem to be relevant for the analysis

In [25]:
df_clean['investigator_or_source']

0                                        R. Collier, GSAF
1                          K.McMurray, TrackingSharks.com
2                          K.McMurray, TrackingSharks.com
3                                          B. Myatt, GSAF
4                                               A .Kipper
                              ...                        
5744    L. Becke in New York Sun, 9/9/1906; L. Schultz...
5745                     H. Taunton; N. Bartlett,  p. 234
5746                H. Taunton; N. Bartlett,  pp. 233-234
5747                 F. Schwartz, p.23; C. Creswell, GSAF
5748                                           S.W. Baker
Name: investigator_or_source, Length: 5745, dtype: object

In [26]:
df_clean.drop([4826, 3627], axis=0, inplace=True)
df_clean['investigator_or_source'].fillna('no_source', inplace=True)

#### In Activity column we just find and choose one of the verbs in gerund to describe the scene the victim was doing before being attacked.

In [27]:
def finding_verbs_gerund(string):
    string = str(string)
    if 'ing' in string:
        new_word = re.findall(r"[A-Za-z]+(?:ing)", string)
        if len(new_word) > 1:
            return new_word[-1]
        else: 
            return ''.join(new_word)
    else:
        return string
    
df_clean['activity'] = df_clean['activity'].apply(finding_verbs_gerund)


### The core of the analysis will be the specie of the shark that could be on the attack. Still there are several nullish values, so I decided to name them Unidentified.

In [28]:
df_clean['species'].info()
df_clean['species'].fillna('Unidentified', inplace=True)

<class 'pandas.core.series.Series'>
Int64Index: 5743 entries, 0 to 5748
Series name: species
Non-Null Count  Dtype 
--------------  ----- 
3260 non-null   object
dtypes: object(1)
memory usage: 89.7+ KB


In [29]:
check_nulls()

date                       0.000000
type                       0.034825
country                    0.000000
area                       0.000000
location                   4.910326
activity                   0.000000
name                       0.000000
sex                        0.000000
age                       42.625805
injury                     0.000000
fatal_(y/n)                0.000000
species                    0.000000
investigator_or_source     0.000000
pdf                        0.000000
href_formula               0.017413
href                       0.000000
dtype: float64

## Since our values seems to be the tidest posibble. I decided to rename columns and export the final output.


In [34]:
new_columns_names = {'sex':'gender', 'species': 'shark_species',  'investigator_or_source':'source' , 'fatal_(y/n)':  'is_fatal_attack', 'pdf':'files'  }
df_clean = df_clean.rename(columns=new_columns_names)
df_clean

,date,type,country,area,location,activity,name,gender,age,injury,is_fatal_attack,shark_specie,source,files,href_formula,href
0,2018-06-25,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",False,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2018-06-18,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,False,Unidentified,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2018-06-09,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,False,Unidentified,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2018-06-08,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,<NA>,Minor injury to lower leg,False,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2018-06-04,Provoked,MEXICO,Colima,La Ticla,diving,Gustavo Ramos,M,<NA>,Lacerations to leg & hand shark PROVOKED INCIDENT,False,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5744,1906-01-01,Unprovoked,AUSTRALIA,New South Wales,,Swimming,Arab boy,M,<NA>,FATAL,True,Said to involve a grey nurse shark that leapt ...,"L. Becke in New York Sun, 9/9/1906; L. Schultz...",ND-0006-ArabBoy-Prymount.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
5745,1903-01-01,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,<NA>,FATAL,True,Unidentified,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
5746,1903-01-01,Unprovoked,AUSTRALIA,Western Australia,NaN,diving,Ahmun,M,<NA>,FATAL,True,Unidentified,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
5747,1900-01-01,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,<NA>,FATAL,True,Unidentified,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [31]:
df_clean.to_csv('clean-dataset.csv')